In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from fairdata import FairData

In [2]:
df_raw = pd.read_csv('data/COMPAS/compas-scores-two-years.csv')
df_raw = df_raw.loc[:,[
    'sex', 'race', 'age', 'juv_fel_count','juv_misd_count',
    'juv_other_count', 'priors_count', 'two_year_recid'
]]
df_raw = df_raw.loc[df_raw['race'].isin(['African-American', 'Caucasian', 'Hispanic']), :]

In [3]:
# Encode categorical variables
categorical = ['sex', 'race']
for feature in categorical:
    le = LabelEncoder()
    df_raw[feature] = le.fit_transform(df_raw[feature])
a = df_raw.drop(['sex', 'race', 'two_year_recid'], axis=1)
s = pd.DataFrame({'race-sex': df_raw.race * 2 + df_raw.sex})
y = pd.DataFrame({'two_year_recid': df_raw['two_year_recid']})

In [4]:
# Feature scaling
scaler = StandardScaler()
a = pd.DataFrame(scaler.fit_transform(a), columns=a.columns)
pca = PCA()
a = pd.DataFrame(pca.fit_transform(a), columns=a.columns)
a.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count
0,-0.596519,0.429056,0.061913,0.000674,0.308648
1,0.962933,1.163002,-0.443991,0.970232,-0.761187
2,0.929383,1.031140,-0.217031,-1.391831,0.936699
3,0.548070,-1.669156,-0.304386,-0.308827,-1.258837
4,-0.686701,0.121587,-0.060447,0.156054,0.494724


In [5]:
# Split data into separate training and test set
a_train, a_test, s_train, s_test, y_train, y_test = \
    train_test_split(a, s, y, test_size = 0.25, random_state = 0)

In [6]:
fairdata_ortho = FairData(s_train, a_train, y_train, preprocess_method='o')
fairdata_mdm = FairData(s_train, a_train, y_train, preprocess_method='m')

In [7]:
fairdata_ortho_eval = fairdata_ortho.evaluate(
    a_test, s_test, y_test, metrics=['aa', 'cf', 'mae']
)
pd.DataFrame(fairdata_ortho_eval, index=['AA', 'CF', 'MAE'], columns=['ML', 'FTU', 'FL', 'AA', 'FLAP-1', 'FLAP-2'])

,ML,FTU,FL,AA,FLAP-1,FLAP-2
AA,0.225779,0.145980,0.000000,1.110223e-16,1.110223e-16,0.000000
CF,0.227366,0.140649,0.005434,5.982082e-03,5.845597e-03,0.005434
MAE,0.425570,0.427358,0.440161,4.390602e-01,4.394976e-01,0.440161


In [8]:
fairdata_mdm_eval = fairdata_mdm.evaluate(
    a_test, s_test, y_test, metrics=['aa', 'cf', 'mae']
)
pd.DataFrame(fairdata_mdm_eval, index=['AA', 'CF', 'MAE'], columns=['ML', 'FTU', 'FL', 'AA', 'FLAP-1', 'FLAP-2'])

,ML,FTU,FL,AA,FLAP-1,FLAP-2
AA,0.225779,0.145980,0.000000,1.110223e-16,0.004973,0.005694
CF,0.227366,0.140649,0.005434,5.982082e-03,0.002579,0.002674
MAE,0.425570,0.427358,0.440161,4.390602e-01,0.439288,0.439338


In [1]:
y

NameError: name 'y' is not defined

In [7]:
y_test.shape

(1697, 1)